In [15]:
import numpy as np
import cv2
import glob
import scipy
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw, ImageFont
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import pickle

In [1]:
#Generate lane marker training data

In [121]:
#function to filter the gradient on any or both axises
def mag_thresh(img, sobel_kernel=3,sobel_axis='xy', mag_thresh=(50, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    if sobel_axis != 'xy':
        if sobel_axis == 'x':
            sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
        elif sobel_axis == 'y':
            sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 1, ksize=sobel_kernel)
    # Calculate the absolute value of sobelx
    abs_sobel = np.absolute(sobel)
    # Rescale to 8 bit
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(abs_sobel >= mag_thresh[0]) & (abs_sobel <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

In [122]:
#define the hls channel s threshold function
def hls_select(img, thresh=(100, 255)):
    # 1) Convert to HLS color space
    img_hls = cv2.cvtColor(img,cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    s = np.array(img_hls[:,:,2])
    # 3) Return a binary image of threshold result
    binary_output = np.zeros_like(s)
    binary_output [(s>thresh[0])&(s<=thresh[1])] = 1 # placeholder line
    return binary_output

In [142]:
# four source points for 0:00min - 1:18min
src = np.float32(
[[792,1036],
 [1802,1036],
 [1182,695],
 [1285,695]])

# four desired points
dst = np.float32(
[[330,1080],
 [1350,1080],
 [330,0],
 [1350,0]])
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst,src)

In [144]:
# four source points for 1:18min - 5:00min
src = np.float32(
[[424,1041],
 [1400,1041],
 [900,693],
 [1033,693]])

# four desired points
dst = np.float32(
[[274,1080],
 [1350,1080],
 [274,0],
 [1350,0]])
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst,src)

In [124]:
def warper(img, src, dst):

    # Compute and apply perpective transform
    img_size = (img.shape[1], img.shape[0]) 
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)  # keep same size as input image

    return warped

In [125]:
#warp first then color thresh
def image_warpbinary(image): 
    warp = warper(image,src,dst)
    sx_binary = mag_thresh(warp, sobel_kernel=3,sobel_axis='x', mag_thresh=(60,255))
    s_binary = hls_select(warp, thresh=(80, 255))
    combined_binary = np.zeros_like(sx_binary)
    combined_binary[(s_binary == 1) | (sx_binary == 1)] = 1
    return combined_binary

In [136]:
def find_posxy_fit(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 12
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 100
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    left_fitx = np.int_(left_fitx)
    right_fitx = np.int_(right_fitx)
    #return left_fit,right_fit,left_fitx,right_fitx
    #return left_fit,right_fit
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-50, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+50, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-55, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+55, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    
    #Plot the polynomial lines onto the image
    #plt.plot( left_fitx, ploty,linewidth=7, color='yellow')
    #plt.plot( right_fitx, ploty,linewidth=7, color='yellow')
    #window_img = plt.gcf()
    result = cv2.addWeighted(out_img, 1, window_img, 1, 0)
    return result

In [127]:
#generate ground truth image and visulize
def drawresult(image,lane_warp):
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(lane_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    return result

In [128]:
def marklane(image):
    warp_image = image_warpbinary(image)
    lane = find_posxy_fit(warp_image)
    result = drawresult(image,lane)
    return result

In [129]:
def generate_gt(image):
    warp_image = image_warpbinary(image)
    lane_warp = find_posxy_fit(warp_image)
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(lane_warp, Minv, (image.shape[1], image.shape[0]))
    return newwarp

In [ ]:
#Generate lane line mark video and ground truth

In [ ]:
test_output = './gts79_259.mp4'
clip1 = VideoFileClip('./video 2_processed.mov').subclip(79,259)
white_clip = clip1.fl_image(generate_gt) #NOTE: this function expects color images!!
%time white_clip.write_videofile(test_output, audio=False)

In [ ]:
#Below is generating training data

In [137]:
clip1 = VideoFileClip('./test.mp4').subclip(0,180)
clip1.save_frame('./data/test.jpg',t=12)

In [215]:
clip_combine = VideoFileClip('./test79_259.mp4').subclip(0,300)
clip_gt = VideoFileClip('./gts79_259.mp4').subclip(0,300)
clip_img = VideoFileClip('./video 2_processed.mov').subclip(79,259)
for i in range(174,179):
    comimgpath = './Lanedata/Combine/img{0}.png'.format(i+79)
    gtpath = './Lanedata/GT/gt{0}.png'.format(i+79)
    imagepath = './Lanedata/IMG/images{0}.png'.format(i+79)
    clip_combine.save_frame(comimgpath,t=i)
    clip_gt.save_frame(gtpath,t=i)
    clip_img.save_frame(imagepath,t=i)

In [208]:
i = 121.51
comimgpath = './Lanedata/Combine/img{0}.png'.format(i+79)
gtpath = './Lanedata/GT/gt{0}.png'.format(i+79)
imagepath = './Lanedata/IMG/images{0}.png'.format(i+79)
clip_combine.save_frame(comimgpath,t=i)
clip_gt.save_frame(gtpath,t=i)
clip_img.save_frame(imagepath,t=i)

In [ ]:
def load_data(img_size):
    imagefiles = sorted(glob.glob('./Lanedata/IMG/images*.png'))
    gtfiles = sorted(glob.glob('./Lanedata/GT/gt*.png'))
    images = []
    gts = []
    for image,gt in zip(imagefiles,gtfiles):
        image = cv2.imread(image).astype(np.float32)
        gt = cv2.imread(gt).astype(np.float32)
        images.append(image)
        gts.append(gt)
        images.append(cv2.flip(image, 1))
        gts.append(cv2.flip(gt, 1))
    # prepare the training data
    x_train = np.array(images)
    y_train = np.array(gts)
    background_color = (0,0,0)
    images,gts = [],[]
    for image,gt_image in zip(x_train,y_train):
        image = scipy.misc.imresize(image, img_size)
        gt_image = scipy.misc.imresize(gt_image, img_size)
        #annotate ground truth
        gt_bg = np.all(gt_image == background_color, axis=2)
        gt_bg = gt_bg.reshape(*gt_bg.shape, 1)
        gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)
        images.append(image)
        gts.append(gt_image)

    X_train,Y_train = np.array(images),np.array(gts)
    return X_train,Y_train

In [ ]:
num_class = 2
#custimize the input image shape
img_size = (540,960,3)
X_train,Y_train = load_data(img_size)
pickle.dump(X_train,open('X_lanetrain','wb'))
pickle.dump(Y_train,open('Y_lanetrain','wb'))